<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/Raffinerieproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raffinerieproblem

In [36]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [37]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Indexmengen

In [38]:
J = ['K1','K2','K3', 'K4'] # Menge der Kraftstoffe
I = ['R1','R2','R3','G'] # Menge der Produktionsressourcen

## Entscheidungsvariablen

In [39]:
infinity = solver.infinity()

In [40]:
J

['K1', 'K2', 'K3', 'K4']

In [41]:
# Definition der Entscheidungsvariablen
x={}
for j in J:
    x[j] = solver.NumVar(0.0, infinity, j)
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 4


In [42]:
x

{'K1': K1, 'K2': K2, 'K3': K3, 'K4': K4}

## Parameter

In [43]:
c={} # Deckungsbeiträge
c['K1']=5
c['K2']=8
c['K3']=15
c['K4']=3

In [44]:
c

{'K1': 5, 'K2': 8, 'K3': 15, 'K4': 3}

In [45]:
a={} # Produktionskoeffizienten, a[ij]
a['R1']={'K1':1,'K2':1, 'K3':2,'K4':1}
a['R2']={'K1':2,'K2':1, 'K3':3,'K4':0}
a['R3']={'K1':0,'K2':1, 'K3':2,'K4':1}
a['G']={'K1':1,'K2':1, 'K3':1,'K4':1}

In [46]:
a

{'R1': {'K1': 1, 'K2': 1, 'K3': 2, 'K4': 1},
 'R2': {'K1': 2, 'K2': 1, 'K3': 3, 'K4': 0},
 'R3': {'K1': 0, 'K2': 1, 'K3': 2, 'K4': 1},
 'G': {'K1': 1, 'K2': 1, 'K3': 1, 'K4': 1}}

In [47]:
b={} # Kapazitäten Produktionsfaktoren
b['R1']=1000
b['R2']=500
b['R3']=1500
b['G']=2250

In [48]:
b

{'R1': 1000, 'R2': 500, 'R3': 1500, 'G': 2250}

## Zielfunktion

In [49]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(c[j]*x[j] for j in J))

## Nebenbedingungen

In [50]:
I

['R1', 'R2', 'R3', 'G']

In [51]:
for i in I:
    solver.Add(sum(a[i][j]*x[j] for j in J)<=b[i])
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

Anzahl Nebenbedingungen = 4


## Berechnung Lösung

In [52]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for j in J:
        print(f'{j} =', round(x[j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 5500.0
K1 = 0
K2 = 500
K3 = 0
K4 = 500
